In [ ]:
# adapted from this medium article: https://medium.com/analytics-vidhya/how-to-create-a-choropleth-map-using-uber-h3-plotly-python-458f51593548

In [1]:
# Import dependencies
import h3
import geopandas as gpd
import numpy as np

In [4]:
import geopandas as gpd

url = "https://mapservices.gov.yk.ca/arcgis/rest/services/GeoYukon/GY_EmergencyManagement/MapServer/8/query?outFields=*&where=1%3D1&f=geojson"
fire_ignitions = gpd.read_file(url)

fire_ignitions.describe()

,FIRE_NUMBER,AREA_HECTARES,LATITUDE_DD,LONGITUDE_DD,OBJECTID,FIRE_YEAR,DECADE,FIRE_INITIAL_DATE,FIRE_EXTINGUISH_DATE
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,2.910000e+02,2.520000e+02
mean,15.067000,1466.996690,62.458597,-135.366281,506.661000,2101.966000,1985.470000,1.392321e+12,1.381437e+12
std,13.919564,7740.848897,1.948432,3.438055,299.654008,940.091144,19.904152,2.065345e+11,2.017693e+11
min,1.000000,0.000000,59.703920,-141.000000,1.000000,1946.000000,1940.000000,1.052156e+12,1.087987e+12
25%,5.000000,0.010000,60.733290,-137.993700,250.750000,1975.000000,1970.000000,1.214672e+12,1.214851e+12
50%,11.000000,0.805000,62.193665,-135.273195,500.500000,1991.000000,1990.000000,1.376404e+12,1.375920e+12
75%,22.000000,100.000000,63.741665,-133.876675,750.250000,2006.000000,2000.000000,1.590667e+12,1.529906e+12
max,97.000000,128717.000000,68.697510,-124.033330,1101.000000,9990.000000,2020.000000,1.693181e+12,1.701734e+12


In [5]:
numeric_columns = (fire_ignitions
                   .select_dtypes(include=np.number)
                   .columns
                   .to_list())
fire_ignitions[numeric_columns].head()

,FIRE_NUMBER,AREA_HECTARES,LATITUDE_DD,LONGITUDE_DD,OBJECTID,FIRE_YEAR,DECADE,FIRE_INITIAL_DATE,FIRE_EXTINGUISH_DATE
0,15,3.00,60.900030,-129.400000,32,2012,2010,1.343410e+12,1.343347e+12
1,40,0.04,63.564880,-135.792580,33,2015,2010,1.435247e+12,1.435346e+12
2,26,8.70,60.577487,-134.949146,34,2023,2020,1.692749e+12,1.693440e+12
3,5,0.02,62.150000,-133.199720,35,1975,1970,NaN,NaN
4,14,0.60,60.493060,-134.305000,36,1955,1950,NaN,NaN


In [12]:
if 'LATITUDE_D' in fire_ignitions.columns and 'LONGITUDE_' in fire_ignitions.columns:
    fire_ignitions = fire_ignitions[['FIRE_ID', 'LATITUDE_D', 'LONGITUDE_']]
else:
    print("Columns 'LATITUDE_D' and 'LONGITUDE_' do not exist in the dataframe.")


Columns 'LATITUDE_D' and 'LONGITUDE_' do not exist in the dataframe.


In [8]:
# Import h3 and map ignitions to h3 hexagons
H3_res = 5
def geo_to_h3(row):
    return h3.geo_to_h3(lat=row.lat,lng=row.lng,resolution = H3_res)

In [13]:
# Define the geo_to_h3 function
def geo_to_h3(row):
    return h3.geo_to_h3(lat=row.LATITUDE_DD, lng=row.LONGITUDE_DD, resolution=H3_res)

# Apply the geo_to_h3 function to the fire_ignitions dataframe
fire_ignitions['h3_cell'] = fire_ignitions.apply(geo_to_h3, axis=1)

In [15]:
# Import the missing column
fire_ignitions['ignition_id'] = fire_ignitions['FIRE_ID']

# Group by 'h3_cell' and aggregate 'ignition_id' as a list
fire_ignitions_g = (fire_ignitions
                    .groupby('h3_cell')
                    .ignition_id
                    .agg(list)
                    .to_frame("ids")
                    .reset_index())

# Count the number of points inside each hexagon
fire_ignitions_g['count'] = (fire_ignitions_g['ids']
                             .apply(lambda ignition_ids: len(ignition_ids)))


In [16]:
fire_ignitions_g.sort_values('count', ascending=False)

,h3_cell,ids,count
106,85138673fffffff,"[2012XY007, 2013XY011, 2010XY002, 2023XY012, 2...",10
149,8513a60ffffffff,"[2015MA040, 2006MA004, 2012MA030, 2012MA028]",4
112,85138c77fffffff,"[2016WL005, 2015WL012, 2022WL024]",3
200,8513b493fffffff,"[2007DA030, 2015DA033, 2004DA025]",3
104,85138657fffffff,"[2016XY003, 2017XY009, 2010XY004]",3
...,...,...,...
74,850d6e37fffffff,[2008OC021],1
75,8512304bfffffff,[2012WL014],1
77,85123667fffffff,[2015WL006],1
78,8512366ffffffff,[2004WL028],1


In [17]:
from shapely.geometry import Polygon


def add_geometry(row):
  points = h3.h3_to_geo_boundary(row['h3_cell'], True)
  return Polygon(points)


# Apply function into our dataframe
fire_ignitions_g['geometry'] = (fire_ignitions_g
                                .apply(add_geometry, axis=1))

In [22]:
from geojson import Feature, Point, FeatureCollection, Polygon


def hexagons_dataframe_to_geojson(df_hex, hex_id_field, geometry_field, value_field, file_output=None):

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry=row[geometry_field],
                          id=row[hex_id_field],
                          properties={"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else:
      return feat_collection

In [28]:
geojson_obj = (hexagons_dataframe_to_geojson
               (fire_ignitions_grouped,
                hex_id_field='h3_cell',
                value_field='count',
                geometry_field='geometry'))

TypeError: hexagons_dataframe_to_geojson() got an unexpected keyword argument 'fire_ignitions_grouped'

In [26]:
import plotly.express as px

In [ ]:
fig = (px.choropleth_mapbox(
    fire_ignitions_g,
    geojson=geojson_obj,
    locations='h3_cell',
    color='count',
    color_continuous_scale="Viridis",
    range_color=(0, fire_ignitions_g['count'].mean()),                  mapbox_style='carto-positron',
    zoom=7,
    center={"lat": 65.469211, "lon": -136.713865},
    opacity=0.7,
    labels={'count': '# of fire ignitions '}))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [29]:
fire_ignitions_g.to_csv('fire_ignitions_g.csv', index=False)